In [1]:
using Pkg
Pkg.activate(".")

In [2]:
using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames

In [ ]:
include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions.jl")

In [ ]:
include("./LShaped/src/LShaped.jl")

In [ ]:
exmodel = JuMP.read_from_file("./ef_12scen_test.mps")

In [68]:
get_PR_lb(exmodel,101,0)

PR_lb_{101}_0 : PR_{101}_0 - 0.2 ER_{101}_0 >= 0.0

In [69]:
get_ER_lb(exmodel,101,0)

ER_lb_{101}_0 : ER_{101}_0 >= 0.0

In [70]:
get_ER_ub(exmodel,101,0)

In [71]:
get_PR_ub(exmodel,101,0)

PR_ub_{101}_0 : PR_{101}_0 - 0.8 ER_{101}_0 <= 0.0

In [67]:
for bus in buses
    oldPRlb = get_PR_lb(exmodel,bus,0)
    oldERlb = get_ER_lb(exmodel,bus,0)
    oldERub = get_ER_ub(exmodel,bus,0)
    
    JuMP.delete(exmodel,oldPRlb)
    JuMP.unregister(exmodel,Symbol(name(oldPRlb)))

    JuMP.delete(exmodel,oldERlb)
    JuMP.unregister(exmodel,Symbol(name(oldERlb)))
    
    JuMP.delete(exmodel,oldERub)
    JuMP.unregister(exmodel,Symbol(name(oldERub)))
    
    varER = get_ER_variable(exmodel,bus,0)
    varPR = get_PR_variable(exmodel,bus,0)
    
    JuMP.@constraint(exmodel, varER >= 0.0, base_name = "ER_lb_{$(bus)}_0")
    JuMP.@constraint(exmodel, varPR >= 0.2*varER, base_name = "PR_lb_{$(bus)}_0")
    JuMP.@constraint(exmodel, varPR <= 0.8*varER, base_name = "PR_ub_{$(bus)}_0")
end

In [72]:
con = get_expansion_budget(exmodel,0)
for bus in buses
    set_normalized_coefficient(con, get_PR_variable(exmodel,bus,0), 56.12)
    set_normalized_coefficient(con, get_ER_variable(exmodel,bus,0), 0.3)
end

con

expansion_budget_0 : 56.12 PR_{101}_0 + 0.3 ER_{101}_0 + 56.12 PR_{102}_0 + 0.3 ER_{102}_0 + 56.12 PR_{103}_0 + 0.3 ER_{103}_0 + 56.12 PR_{104}_0 + 0.3 ER_{104}_0 + 56.12 PR_{105}_0 + 0.3 ER_{105}_0 + 56.12 PR_{106}_0 + 0.3 ER_{106}_0 + 56.12 PR_{107}_0 + 0.3 ER_{107}_0 + 56.12 PR_{108}_0 + 0.3 ER_{108}_0 + 56.12 PR_{109}_0 + 0.3 ER_{109}_0 + 56.12 PR_{110}_0 + 0.3 ER_{110}_0 + 56.12 PR_{111}_0 + 0.3 ER_{111}_0 + 56.12 PR_{112}_0 + 0.3 ER_{112}_0 + 56.12 PR_{113}_0 + 0.3 ER_{113}_0 + 56.12 PR_{114}_0 + 0.3 ER_{114}_0 + 56.12 PR_{115}_0 + 0.3 ER_{115}_0 + 56.12 PR_{116}_0 + 0.3 ER_{116}_0 + 56.12 PR_{117}_0 + 0.3 ER_{117}_0 + 56.12 PR_{118}_0 + 0.3 ER_{118}_0 + 56.12 PR_{119}_0 + 0.3 ER_{119}_0 + 56.12 PR_{120}_0 + 0.3 ER_{120}_0 + 56.12 PR_{121}_0 + 0.3 ER_{121}_0 + 56.12 PR_{122}_0 + 0.3 ER_{122}_0 + 56.12 PR_{123}_0 + 0.3 ER_{123}_0 + 56.12 PR_{124}_0 + 0.3 ER_{124}_0 <= 200.0

In [73]:
#change loss of load cost to reflect ERCOT 8500/MWh
for scen = 1:12
    for bus in buses
        for ts in timesteps
            var = get_lossofload_variable(exmodel,bus,ts,scen)
            JuMP.set_objective_coefficient(exmodel, var, 8500.0*100*365.25/1000000/12)
        end
    end
end

In [74]:
for (var, coeff) in objective_function(exmodel).terms
    println(var, " ", coeff)
end

PR_{101}_0 56.11999999999999
ER_{101}_0 0.29999999999999993
PR_{102}_0 56.11999999999999
ER_{102}_0 0.29999999999999993
PR_{103}_0 56.11999999999999
ER_{103}_0 0.29999999999999993
PR_{104}_0 56.11999999999999
ER_{104}_0 0.29999999999999993
PR_{105}_0 56.11999999999999
ER_{105}_0 0.29999999999999993
PR_{106}_0 56.11999999999999
ER_{106}_0 0.29999999999999993
PR_{107}_0 56.11999999999999
ER_{107}_0 0.29999999999999993
PR_{108}_0 56.11999999999999
ER_{108}_0 0.29999999999999993
PR_{109}_0 56.11999999999999
ER_{109}_0 0.29999999999999993
PR_{110}_0 56.11999999999999
ER_{110}_0 0.29999999999999993
PR_{111}_0 56.11999999999999
ER_{111}_0 0.29999999999999993
PR_{112}_0 56.11999999999999
ER_{112}_0 0.29999999999999993
PR_{113}_0 56.11999999999999
ER_{113}_0 0.29999999999999993
PR_{114}_0 56.11999999999999
ER_{114}_0 0.29999999999999993
PR_{115}_0 56.11999999999999
ER_{115}_0 0.29999999999999993
PR_{116}_0 56.11999999999999
ER_{116}_0 0.29999999999999993
PR_{117}_0 56.11999999999999
ER_{117}_0 

Pth_{107_CC_1,15}_1 0.08263868937393751
Pth_{113_CT_1,15}_1 0.10477880641753125
Pth_{113_CT_2,15}_1 0.10477880641753125
Pth_{113_CT_3,15}_1 0.10477880641753125
Pth_{113_CT_4,15}_1 0.10477880641753125
Pth_{115_STEAM_1,15}_1 0.428145654616875
Pth_{115_STEAM_2,15}_1 0.428145654616875
Pth_{115_STEAM_3,15}_1 0.07163481047681247
Pth_{116_STEAM_1,15}_1 0.07078224491603125
Pth_{118_CC_1,15}_1 0.08446669102162499
Pth_{123_STEAM_2,15}_1 0.07504185549140625
Pth_{123_STEAM_3,15}_1 0.06898703139559373
Pth_{123_CT_1,15}_1 0.10396241733215625
Pth_{123_CT_4,15}_1 0.10396241733215625
Pth_{123_CT_5,15}_1 0.10396241733215625
Pth_{121_NUCLEAR_1,15}_1 0.024665028125000002
Pth_{101_CT_1,16}_1 0.33389470375687497
Pth_{101_CT_2,16}_1 0.33389470375687497
Pth_{101_STEAM_3,16}_1 0.058810937551249984
Pth_{101_STEAM_4,16}_1 0.058810937551249984
Pth_{102_CT_1,16}_1 0.3307997318578125
Pth_{102_CT_2,16}_1 0.3307997318578125
Pth_{102_STEAM_3,16}_1 0.06569098076532812
Pth_{102_STEAM_4,16}_1 0.06569098076532812
Pth_{107

Pdis_{113_STORAGE_1,3}_1 0.004565624999999999
Pdis_{114_STORAGE_1,3}_1 0.004565624999999999
Pdis_{115_STORAGE_1,3}_1 0.004565624999999999
Pdis_{116_STORAGE_1,3}_1 0.004565624999999999
Pdis_{117_STORAGE_1,3}_1 0.004565624999999999
Pdis_{118_STORAGE_1,3}_1 0.004565624999999999
Pdis_{119_STORAGE_1,3}_1 0.004565624999999999
Pdis_{120_STORAGE_1,3}_1 0.004565624999999999
Pdis_{121_STORAGE_1,3}_1 0.004565624999999999
Pdis_{122_STORAGE_1,3}_1 0.004565624999999999
Pdis_{123_STORAGE_1,3}_1 0.004565624999999999
Pdis_{124_STORAGE_1,3}_1 0.004565624999999999
Pdis_{101_STORAGE_1,4}_1 0.004565624999999999
Pdis_{102_STORAGE_1,4}_1 0.004565624999999999
Pdis_{103_STORAGE_1,4}_1 0.004565624999999999
Pdis_{104_STORAGE_1,4}_1 0.004565624999999999
Pdis_{105_STORAGE_1,4}_1 0.004565624999999999
Pdis_{106_STORAGE_1,4}_1 0.004565624999999999
Pdis_{107_STORAGE_1,4}_1 0.004565624999999999
Pdis_{108_STORAGE_1,4}_1 0.004565624999999999
Pdis_{109_STORAGE_1,4}_1 0.004565624999999999
Pdis_{110_STORAGE_1,4}_1 0.0045656

Pdis_{109_STORAGE_1,11}_1 0.004565624999999999
Pdis_{110_STORAGE_1,11}_1 0.004565624999999999
Pdis_{111_STORAGE_1,11}_1 0.004565624999999999
Pdis_{112_STORAGE_1,11}_1 0.004565624999999999
Pdis_{113_STORAGE_1,11}_1 0.004565624999999999
Pdis_{114_STORAGE_1,11}_1 0.004565624999999999
Pdis_{115_STORAGE_1,11}_1 0.004565624999999999
Pdis_{116_STORAGE_1,11}_1 0.004565624999999999
Pdis_{117_STORAGE_1,11}_1 0.004565624999999999
Pdis_{118_STORAGE_1,11}_1 0.004565624999999999
Pdis_{119_STORAGE_1,11}_1 0.004565624999999999
Pdis_{120_STORAGE_1,11}_1 0.004565624999999999
Pdis_{121_STORAGE_1,11}_1 0.004565624999999999
Pdis_{122_STORAGE_1,11}_1 0.004565624999999999
Pdis_{123_STORAGE_1,11}_1 0.004565624999999999
Pdis_{124_STORAGE_1,11}_1 0.004565624999999999
Pdis_{101_STORAGE_1,12}_1 0.004565624999999999
Pdis_{102_STORAGE_1,12}_1 0.004565624999999999
Pdis_{103_STORAGE_1,12}_1 0.004565624999999999
Pdis_{104_STORAGE_1,12}_1 0.004565624999999999
Pdis_{105_STORAGE_1,12}_1 0.004565624999999999
Pdis_{106_STO

Pdis_{111_STORAGE_1,20}_1 0.004565624999999999
Pdis_{112_STORAGE_1,20}_1 0.004565624999999999
Pdis_{113_STORAGE_1,20}_1 0.004565624999999999
Pdis_{114_STORAGE_1,20}_1 0.004565624999999999
Pdis_{115_STORAGE_1,20}_1 0.004565624999999999
Pdis_{116_STORAGE_1,20}_1 0.004565624999999999
Pdis_{117_STORAGE_1,20}_1 0.004565624999999999
Pdis_{118_STORAGE_1,20}_1 0.004565624999999999
Pdis_{119_STORAGE_1,20}_1 0.004565624999999999
Pdis_{120_STORAGE_1,20}_1 0.004565624999999999
Pdis_{121_STORAGE_1,20}_1 0.004565624999999999
Pdis_{122_STORAGE_1,20}_1 0.004565624999999999
Pdis_{123_STORAGE_1,20}_1 0.004565624999999999
Pdis_{124_STORAGE_1,20}_1 0.004565624999999999
Pdis_{101_STORAGE_1,21}_1 0.004565624999999999
Pdis_{102_STORAGE_1,21}_1 0.004565624999999999
Pdis_{103_STORAGE_1,21}_1 0.004565624999999999
Pdis_{104_STORAGE_1,21}_1 0.004565624999999999
Pdis_{105_STORAGE_1,21}_1 0.004565624999999999
Pdis_{106_STORAGE_1,21}_1 0.004565624999999999
Pdis_{107_STORAGE_1,21}_1 0.004565624999999999
Pdis_{108_STO

...

OL_{107,10}_6 6.0874999999999995
OL_{108,10}_6 6.0874999999999995
OL_{109,10}_6 6.0874999999999995
OL_{110,10}_6 6.0874999999999995
OL_{111,10}_6 6.0874999999999995
OL_{112,10}_6 6.0874999999999995
OL_{113,10}_6 6.0874999999999995
OL_{114,10}_6 6.0874999999999995
OL_{115,10}_6 6.0874999999999995
OL_{116,10}_6 6.0874999999999995
OL_{117,10}_6 6.0874999999999995
OL_{118,10}_6 6.0874999999999995
OL_{119,10}_6 6.0874999999999995
OL_{120,10}_6 6.0874999999999995
OL_{121,10}_6 6.0874999999999995
OL_{122,10}_6 6.0874999999999995
OL_{123,10}_6 6.0874999999999995
OL_{124,10}_6 6.0874999999999995
OL_{101,11}_6 6.0874999999999995
OL_{102,11}_6 6.0874999999999995
OL_{103,11}_6 6.0874999999999995
OL_{104,11}_6 6.0874999999999995
OL_{105,11}_6 6.0874999999999995
OL_{106,11}_6 6.0874999999999995
OL_{107,11}_6 6.0874999999999995
OL_{108,11}_6 6.0874999999999995
OL_{109,11}_6 6.0874999999999995
OL_{110,11}_6 6.0874999999999995
OL_{111,11}_6 6.0874999999999995
OL_{112,11}_6 6.0874999999999995
OL_{113,11

OL_{109,23}_6 6.0874999999999995
OL_{110,23}_6 6.0874999999999995
OL_{111,23}_6 6.0874999999999995
OL_{112,23}_6 6.0874999999999995
OL_{113,23}_6 6.0874999999999995
OL_{114,23}_6 6.0874999999999995
OL_{115,23}_6 6.0874999999999995
OL_{116,23}_6 6.0874999999999995
OL_{117,23}_6 6.0874999999999995
OL_{118,23}_6 6.0874999999999995
OL_{119,23}_6 6.0874999999999995
OL_{120,23}_6 6.0874999999999995
OL_{121,23}_6 6.0874999999999995
OL_{122,23}_6 6.0874999999999995
OL_{123,23}_6 6.0874999999999995
OL_{124,23}_6 6.0874999999999995
OL_{101,24}_6 6.0874999999999995
OL_{102,24}_6 6.0874999999999995
OL_{103,24}_6 6.0874999999999995
OL_{104,24}_6 6.0874999999999995
OL_{105,24}_6 6.0874999999999995
OL_{106,24}_6 6.0874999999999995
OL_{107,24}_6 6.0874999999999995
OL_{108,24}_6 6.0874999999999995
OL_{109,24}_6 6.0874999999999995
OL_{110,24}_6 6.0874999999999995
OL_{111,24}_6 6.0874999999999995
OL_{112,24}_6 6.0874999999999995
OL_{113,24}_6 6.0874999999999995
OL_{114,24}_6 6.0874999999999995
OL_{115,24

Pth_{102_STEAM_3,10}_7 0.06569098076532812
Pth_{102_STEAM_4,10}_7 0.06569098076532812
Pth_{107_CC_1,10}_7 0.08263868937393751
Pth_{113_CT_1,10}_7 0.10477880641753125
Pth_{113_CT_2,10}_7 0.10477880641753125
Pth_{113_CT_3,10}_7 0.10477880641753125
Pth_{113_CT_4,10}_7 0.10477880641753125
Pth_{115_STEAM_1,10}_7 0.428145654616875
Pth_{115_STEAM_2,10}_7 0.428145654616875
Pth_{115_STEAM_3,10}_7 0.07163481047681247
Pth_{116_STEAM_1,10}_7 0.07078224491603125
Pth_{118_CC_1,10}_7 0.08446669102162499
Pth_{123_STEAM_2,10}_7 0.07504185549140625
Pth_{123_STEAM_3,10}_7 0.06898703139559373
Pth_{123_CT_1,10}_7 0.10396241733215625
Pth_{123_CT_4,10}_7 0.10396241733215625
Pth_{123_CT_5,10}_7 0.10396241733215625
Pth_{121_NUCLEAR_1,10}_7 0.024665028125000002
Pth_{101_CT_1,11}_7 0.33389470375687497
Pth_{101_CT_2,11}_7 0.33389470375687497
Pth_{101_STEAM_3,11}_7 0.058810937551249984
Pth_{101_STEAM_4,11}_7 0.058810937551249984
Pth_{102_CT_1,11}_7 0.3307997318578125
Pth_{102_CT_2,11}_7 0.3307997318578125
Pth_{102

Excessive output truncated after 524291 bytes.


Pth_{115_STEAM_3,18}_7 0.07163481047681247
Pth_{116_STEAM_1,18}_7 0.07078224491603125
Pth_{118_CC_1,18}_7 0.08446669102162499
Pth_{123_STEAM_2,18}_7 0.07504185549140625
Pth_{123_STEAM_3,18}_7 0.06898703139559373
Pth_{123_CT_1,18}_7 0.10396241733215625
Pth_{123_CT_4,18}_7 0.10396241733215625
Pth_{123_CT_5,18}_7 0.10396241733215625
Pth_{121_NUCLEAR_1,18}_7 0.024665028125000002
Pth_{101_CT_1,19}_7 0.33389470375687497
Pth_{101_CT_2,19}_7 0.33389470375687497
Pth_{101_STEAM_3,19}_7 0.058810937551249984
Pth_{101_STEAM_4,19}_7 0.058810937551249984
Pth_{102_CT_1,19}_7 0.3307997318578125
Pth_{102_CT_2,19}_7 0.3307997318578125
Pth_{102_STEAM_3,19}_7 0.06569098076532812
Pth_{102_STEAM_4,19}_7 0.06569098076532812
Pth_{107_CC_1,19}_7 0.08263868937393751
Pth_{113_CT_1,19}_7 0.10477880641753125
Pth_{113_CT_2,19}_7 0.10477880641753125
Pth_{113_CT_3,19}_7 0.10477880641753125
Pth_{113_CT_4,19}_7 0.10477880641753125
Pth_{115_STEAM_1,19}_7 0.428145654616875
Pth_{115_STEAM_2,19}_7 0.428145654616875
Pth_{11

In [75]:
JuMP.set_optimizer(exmodel, Gurobi.Optimizer)
JuMP.set_optimizer_attribute(exmodel, "OutputFlag", 1) 
JuMP.set_optimizer_attribute(exmodel, "Method", 1) 

JuMP.optimize!(exmodel)

Academic license - for non-commercial use only - expires 2022-08-19
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 111817 rows, 70608 columns and 1439880 nonzeros
Model fingerprint: 0x08585b84
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [5e-03, 6e+01]
  Bounds range     [1e-01, 5e+00]
  RHS range        [1e-03, 2e+02]
Presolve removed 68916 rows and 21656 columns
Presolve time: 0.63s
Presolved: 42901 rows, 49780 columns, 1007088 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          1s
    8332    5.5999434e+02   2.015834e+08   0.000000e+00      5s
   13672    6.3067574e+02   7.137624e+05   0.000000e+00     10s
   18780    6.7100195e+02   1.080344e+05   0.000000e+00     15s
   24473    6.832333

In [76]:
for bus in buses
    varPR = get_PR_variable(exmodel, bus,0)
    varER = get_ER_variable(exmodel, bus,0)
    println(varPR, " ", value(varPR))
    println(varER, " ", value(varER))
end

PR_{101}_0 0.0
ER_{101}_0 0.0
PR_{102}_0 0.0
ER_{102}_0 0.0
PR_{103}_0 0.0
ER_{103}_0 0.0
PR_{104}_0 0.0
ER_{104}_0 0.0
PR_{105}_0 0.0
ER_{105}_0 0.0
PR_{106}_0 0.0
ER_{106}_0 0.0
PR_{107}_0 0.0
ER_{107}_0 0.0
PR_{108}_0 0.0
ER_{108}_0 0.0
PR_{109}_0 0.0
ER_{109}_0 0.0
PR_{110}_0 0.0
ER_{110}_0 0.0
PR_{111}_0 0.0
ER_{111}_0 0.0
PR_{112}_0 0.0
ER_{112}_0 0.0
PR_{113}_0 0.0
ER_{113}_0 0.0
PR_{114}_0 0.2970924180099013
ER_{114}_0 1.4854620900495066
PR_{115}_0 0.0
ER_{115}_0 0.0
PR_{116}_0 1.6240825598620396
ER_{116}_0 8.120412799310198
PR_{117}_0 1.469783051206798
ER_{117}_0 7.348915256033989
PR_{118}_0 0.0
ER_{118}_0 0.0
PR_{119}_0 0.0
ER_{119}_0 0.0
PR_{120}_0 0.0
ER_{120}_0 0.0
PR_{121}_0 0.0
ER_{121}_0 0.0
PR_{122}_0 0.08005897890460051
ER_{122}_0 0.40029489452300254
PR_{123}_0 0.0
ER_{123}_0 0.0
PR_{124}_0 0.0
ER_{124}_0 0.0


In [77]:
JuMP.write_to_file(exmodel, "./ef_12scen_test_revised.mps")

In [83]:
model1 = JuMP.read_from_file("./storage_expansion/first_stage/first_stage_model.mps")

A JuMP Model
Minimization problem with:
Variables: 48
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 48 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [84]:
for bus in buses
    oldPRlb = get_PR_lb(model1,bus)
    oldERlb = get_ER_lb(model1,bus)
    oldERub = get_ER_ub(model1,bus)
    
    JuMP.delete(model1,oldPRlb)
    JuMP.unregister(model1,Symbol(name(oldPRlb)))

    JuMP.delete(model1,oldERlb)
    JuMP.unregister(model1,Symbol(name(oldERlb)))
    
    JuMP.delete(model1,oldERub)
    JuMP.unregister(model1,Symbol(name(oldERub)))
    
    varER = get_ER_variable(model1,bus)
    varPR = get_PR_variable(model1,bus)
    
    JuMP.@constraint(model1, varER >= 0.0, base_name = "ER_lb_{$(bus)}")
    JuMP.@constraint(model1, varPR >= 0.2*varER, base_name = "PR_lb_{$(bus)}")
    JuMP.@constraint(model1, varPR <= 0.8*varER, base_name = "PR_ub_{$(bus)}")
end

In [85]:
con = get_expansion_budget(model1)
for bus in buses
    set_normalized_coefficient(con, get_PR_variable(model1,bus), 56.12)
    set_normalized_coefficient(con, get_ER_variable(model1,bus), 0.3)
end

con

expansion_budget : 56.12 PR_{101} + 0.3 ER_{101} + 56.12 PR_{102} + 0.3 ER_{102} + 56.12 PR_{103} + 0.3 ER_{103} + 56.12 PR_{104} + 0.3 ER_{104} + 56.12 PR_{105} + 0.3 ER_{105} + 56.12 PR_{106} + 0.3 ER_{106} + 56.12 PR_{107} + 0.3 ER_{107} + 56.12 PR_{108} + 0.3 ER_{108} + 56.12 PR_{109} + 0.3 ER_{109} + 56.12 PR_{110} + 0.3 ER_{110} + 56.12 PR_{111} + 0.3 ER_{111} + 56.12 PR_{112} + 0.3 ER_{112} + 56.12 PR_{113} + 0.3 ER_{113} + 56.12 PR_{114} + 0.3 ER_{114} + 56.12 PR_{115} + 0.3 ER_{115} + 56.12 PR_{116} + 0.3 ER_{116} + 56.12 PR_{117} + 0.3 ER_{117} + 56.12 PR_{118} + 0.3 ER_{118} + 56.12 PR_{119} + 0.3 ER_{119} + 56.12 PR_{120} + 0.3 ER_{120} + 56.12 PR_{121} + 0.3 ER_{121} + 56.12 PR_{122} + 0.3 ER_{122} + 56.12 PR_{123} + 0.3 ER_{123} + 56.12 PR_{124} + 0.3 ER_{124} <= 200.0

In [86]:
JuMP.write_to_file(model1,"./storage_expansion_revised/first_stage/first_stage_model.mps")

In [88]:
for i = 1:12
    model = JuMP.read_from_file("./storage_expansion/second_stage/scen_$(i).mps")
    
    for bus in buses
        for ts in timesteps
            var = get_lossofload_variable(model,bus,ts)
            JuMP.set_objective_coefficient(model, var, 8500.0*100*365.25/1000000)
        end
    end
    
    JuMP.write_to_file(model,"./storage_expansion_revised/second_stage/scen_$(i).mps")
end

In [89]:
vardict = Dict{Int64,Array{Any}}()

modelvars = JuMP.read_from_file("./storage_expansion_revised/second_stage/scen_1.mps")

firstvars = []
for bus in buses
    push!(firstvars,(string(get_PR_variable(modelvars,bus)), 0.0, Inf, 0.0))
    push!(firstvars,(string(get_ER_variable(modelvars,bus)), 0.0, Inf, 0.0))
end

vardict[1] = firstvars

secondvars = [];

for ts in timesteps
    
    for gen in gens
        push!(secondvars,string(get_thermal_variable(modelvars,gen,ts)))
    end
    for br in branches
        push!(secondvars,string(get_line_variable(modelvars,br,ts)))
    end
    
    push!(secondvars, string(get_wind_variable(modelvars,122,ts)))
    
    for bus in buses
        push!(secondvars, string(get_charging_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_discharging_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_stored_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_lossofload_variable(modelvars,bus,ts)))
        push!(secondvars, string(get_overload_variable(modelvars,bus,ts)))
    end
end

vardict[2] = secondvars

4392-element Vector{Any}:
 "Pth_{101_CT_1,1}"
 "Pth_{101_CT_2,1}"
 "Pth_{101_STEAM_3,1}"
 "Pth_{101_STEAM_4,1}"
 "Pth_{102_CT_1,1}"
 "Pth_{102_CT_2,1}"
 "Pth_{102_STEAM_3,1}"
 "Pth_{102_STEAM_4,1}"
 "Pth_{107_CC_1,1}"
 "Pth_{113_CT_1,1}"
 "Pth_{113_CT_2,1}"
 "Pth_{113_CT_3,1}"
 "Pth_{113_CT_4,1}"
 ⋮
 "LOL_{122,24}"
 "OL_{122,24}"
 "Pch_{123_STORAGE_1,24}"
 "Pdis_{123_STORAGE_1,24}"
 "Est_{123_STORAGE_1,24}"
 "LOL_{123,24}"
 "OL_{123,24}"
 "Pch_{124_STORAGE_1,24}"
 "Pdis_{124_STORAGE_1,24}"
 "Est_{124_STORAGE_1,24}"
 "LOL_{124,24}"
 "OL_{124,24}"

In [93]:
function first_func()
    
    model = JuMP.read_from_file("./storage_expansion_revised/first_stage/first_stage_model.mps")
    JuMP.set_optimizer(model, Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0) 
    
    return model
end

first_func (generic function with 1 method)

In [94]:
function second_func(scen)
    
    model = JuMP.read_from_file("./storage_expansion_revised/second_stage/scen_$(scen).mps")
    JuMP.set_optimizer(model, Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0) 
    return model
end

second_func (generic function with 1 method)

In [96]:
x, firststage, fs, ittime, niter = LShaped.L_Shaped_Algorithm_new(second_func, 
                            vardict, 
                            12, 
                            first_func, 
                            1e-6, 
                            1000;
                            store = "./L12_revised/");

Academic license - for non-commercial use only - expires 2022-08-19
computing h for subproblem 1...
Initializing subproblem 1...
...making stage_name_idx...
...making variable_info...
Creating subprob[1] struct...
Academic license - for non-commercial use only - expires 2022-08-19
computing h for subproblem 2...
Initializing subproblem 2...
...making stage_name_idx...
...making variable_info...
Creating subprob[2] struct...
Academic license - for non-commercial use only - expires 2022-08-19
computing h for subproblem 3...
Initializing subproblem 3...
...making stage_name_idx...
...making variable_info...
Creating subprob[3] struct...
Academic license - for non-commercial use only - expires 2022-08-19
computing h for subproblem 4...
Initializing subproblem 4...
...making stage_name_idx...
...making variable_info...
Creating subprob[4] struct...
Academic license - for non-commercial use only - expires 2022-08-19
computing h for subproblem 5...
Initializing subproblem 5...
...making stage

Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = -236.45704462661138
w = 1371.1595396859318
Iteration 7/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving du

Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = -235.75018577455376
w = 1371.156461170384
Iteration 15/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving du

Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = -223.1433736688764
w = 1402.644146452821
Iteration 23/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dua

Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = 896.6846242474105
w = 898.4818339984049
Iteration 31/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual

...

...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = 898.1397014848707
w = 898.1572747719234
Iteration 96/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal

...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = 898.1430551506138
w = 898.1532561356969
Iteration 104/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving prima

Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = 898.1443154894307
w = 898.1484636709561
Iteration 112/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dua

Storing Ek and ek...
...saving primal variables...
...saving dual variables...
theta = 898.1472259154771
w = 898.1476030972924
Iteration 128/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dua

theta = 898.1472540890352
w = 898.1472874981303
Iteration 136/1000
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dual variables...
Storing Ek and ek...
...saving primal variables...
...saving dua

In [99]:
for bus in buses
    varPR = get_PR_variable(exmodel, bus,0)
    varER = get_ER_variable(exmodel, bus,0)
    varPRL = get_PR_variable(fs, bus)
    varERL = get_ER_variable(fs, bus)
    println(varPR, " ", value(varPR), " ", value(varPRL))
    println(varER, " ", value(varER), " ", value(varERL))
end

PR_{101}_0 0.0 0.0
ER_{101}_0 0.0 0.0
PR_{102}_0 0.0 0.0
ER_{102}_0 0.0 0.0
PR_{103}_0 0.0 0.0
ER_{103}_0 0.0 0.0
PR_{104}_0 0.0 0.0
ER_{104}_0 0.0 0.0
PR_{105}_0 0.0 0.0
ER_{105}_0 0.0 0.0
PR_{106}_0 0.0 0.0
ER_{106}_0 0.0 0.0
PR_{107}_0 0.0 0.0
ER_{107}_0 0.0 0.0
PR_{108}_0 0.0 0.0
ER_{108}_0 0.0 0.0
PR_{109}_0 0.0 0.0
ER_{109}_0 0.0 0.0
PR_{110}_0 0.0 0.0
ER_{110}_0 0.0 0.0
PR_{111}_0 0.0 0.0
ER_{111}_0 0.0 0.0
PR_{112}_0 0.0 0.0
ER_{112}_0 0.0 0.0
PR_{113}_0 0.0 0.0
ER_{113}_0 0.0 0.0
PR_{114}_0 0.2970924180099013 0.2970924180120908
ER_{114}_0 1.4854620900495066 1.4854620900604538
PR_{115}_0 0.0 0.0
ER_{115}_0 0.0 0.0
PR_{116}_0 1.6240825598620396 1.6240825598821536
ER_{116}_0 8.120412799310198 8.120412799410767
PR_{117}_0 1.469783051206798 1.4697830511851342
ER_{117}_0 7.348915256033989 7.348915255925671
PR_{118}_0 0.0 0.0
ER_{118}_0 0.0 0.0
PR_{119}_0 0.0 0.0
ER_{119}_0 0.0 0.0
PR_{120}_0 0.0 0.0
ER_{120}_0 0.0 0.0
PR_{121}_0 0.0 0.0
ER_{121}_0 0.0 0.0
PR_{122}_0 0.08005897890460